In [42]:
#Import Libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd 
import os

In [1]:
import pandas as pd

# Load only first 500,000 rows to avoid MemoryError
ords_subset = pd.read_pickle(
    r"C:\Users\richa\Documents\01-2026 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merged_updated.pkl"
).iloc[:500000]  # adjust the number of rows depending on your RAM

# Check memory usage
print(ords_subset.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   product_id              500000 non-null  int64   
 1   product_name            500000 non-null  object  
 2   aisle_id                500000 non-null  int64   
 3   department_id           500000 non-null  int64   
 4   prices                  500000 non-null  float64 
 5   order_id                500000 non-null  int64   
 6   user_id                 500000 non-null  int64   
 7   order_number            500000 non-null  int64   
 8   order_dow               500000 non-null  int64   
 9   order_hour_of_day       500000 non-null  int64   
 10  days_since_prior_order  466845 non-null  float64 
 11  add_to_cart_order       500000 non-null  int64   
 12  reordered               500000 non-null  int64   
 13  _merge                  500000 non-null  category
 14  merg

In [2]:
import numpy as np

# Max order per user
ords_subset['max_order'] = ords_subset.groupby('user_id')['order_number'].transform(np.max)

# Mean price per user
ords_subset['mean_price'] = ords_subset.groupby('user_id')['prices'].transform(np.mean)

# Spending flag
ords_subset['spending_flag'] = np.where(ords_subset['mean_price'] < 10, 'Low spender', 'High spender')

# Median days since prior order
ords_subset['median_days_since_prior'] = ords_subset.groupby('user_id')['days_since_prior_order'].transform(np.median)

# Order frequency flag
def frequency_flag(days):
    if days > 20:
        return 'Non-frequent customer'
    elif 10 < days <= 20:
        return 'Regular customer'
    else:
        return 'Frequent customer'

ords_subset['order_frequency_flag'] = ords_subset['median_days_since_prior'].map(frequency_flag)

# Save processed subset to pickle
ords_subset.to_pickle(
    r"C:\Users\richa\Documents\01-2026 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_subset_flags.pkl"
)

print("Subset processed successfully!")


C:\Users\richa\AppData\Local\Temp\ipykernel_23008\917023580.py:4: FutureWarning: The provided callable <function max at 0x000001E772EAF100> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_subset['max_order'] = ords_subset.groupby('user_id')['order_number'].transform(np.max)
C:\Users\richa\AppData\Local\Temp\ipykernel_23008\917023580.py:7: FutureWarning: The provided callable <function mean at 0x000001E772EAFB00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_subset['mean_price'] = ords_subset.groupby('user_id')['prices'].transform(np.mean)
C:\Users\richa\AppData\Local\Temp\ipykernel_23008\917023580.py:13: FutureWarning: The provided callable <function median at 0x000001E772FEAC00> is currently using SeriesGroupBy.median. In a fut

Subset processed successfully!


In [3]:
# Load the pickle file
ords_prods_merge = pd.read_pickle(r'C:\Users\richa\Documents\01-2026 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merged_updated.pkl')

In [5]:
# Check memory usage
print(ords_prods_merge.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 19 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              int64   
 1   product_name            object  
 2   aisle_id                int64   
 3   department_id           int64   
 4   prices                  float64 
 5   order_id                int64   
 6   user_id                 int64   
 7   order_number            int64   
 8   order_dow               int64   
 9   order_hour_of_day       int64   
 10  days_since_prior_order  float64 
 11  add_to_cart_order       int64   
 12  reordered               int64   
 13  _merge                  category
 14  merge_indicator         category
 15  price_range_loc         object  
 16  Busiest day             object  
 17  busiest_days_new        object  
 18  busiest_period_of_day   object  
dtypes: category(2), float64(2), int64(10), object(5)
memory usage: 12.1 GB
None


In [7]:
# ords_prods_merge is already loaded

# 1️⃣ Downcast integer columns
int_cols = ords_prods_merge.select_dtypes(include=['int64']).columns
for col in int_cols:
    ords_prods_merge[col] = pd.to_numeric(ords_prods_merge[col], downcast='unsigned')  # 'signed' if negative values exist

# 2️⃣ Downcast float columns
float_cols = ords_prods_merge.select_dtypes(include=['float64']).columns
for col in float_cols:
    ords_prods_merge[col] = pd.to_numeric(ords_prods_merge[col], downcast='float')

# 3️⃣ Convert repeated object columns to category
obj_cols = ords_prods_merge.select_dtypes(include=['object']).columns
for col in obj_cols:
    if ords_prods_merge[col].nunique() / len(ords_prods_merge) < 0.5:
        ords_prods_merge[col] = ords_prods_merge[col].astype('category')


In [8]:
# 4️⃣ Check new memory usage
print(ords_prods_merge.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 19 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   product_id              uint16  
 1   product_name            category
 2   aisle_id                uint8   
 3   department_id           uint8   
 4   prices                  float32 
 5   order_id                uint32  
 6   user_id                 uint32  
 7   order_number            uint8   
 8   order_dow               uint8   
 9   order_hour_of_day       uint8   
 10  days_since_prior_order  float32 
 11  add_to_cart_order       uint8   
 12  reordered               uint8   
 13  _merge                  category
 14  merge_indicator         category
 15  price_range_loc         category
 16  Busiest day             category
 17  busiest_days_new        category
 18  busiest_period_of_day   category
dtypes: category(7), float32(2), uint16(1), uint32(2), uint8(7)
memory usage: 1.1 GB
No

In [9]:
# 5️⃣ Save downcasted dataset
ords_prods_merge.to_pickle(
    r"C:\Users\richa\Documents\01-2026 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merged_downcasted.pkl"
)

In [12]:
#2 Group by department_id and calculate the mean order_number for the entire dataframe
dept_mean_orders = ords_prods_merge.groupby('department_id')['order_number'].mean()

# result
dept_mean_orders


department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

In [13]:
#3.Department 21 has the highest average and department 5 has the lowest average. deatment 21 receive more order

In [14]:
# 4a Create max_order column for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


C:\Users\richa\AppData\Local\Temp\ipykernel_23008\2486295656.py:2: FutureWarning: The provided callable <function max at 0x000001E772EAF100> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


In [16]:
# verify
ords_prods_merge[['user_id', 'order_number', 'max_order']].head(20)

,user_id,order_number,max_order
0,138,28,32
1,138,30,32
2,709,2,5
3,764,1,3
4,764,3,3
5,777,16,26
6,825,3,9
7,910,12,12
8,1052,10,20
9,1052,15,20


In [17]:
# 4b Assign loyalty categories based on max_order using loc
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [18]:
# verify 20 rows
ords_prods_merge[['user_id', 'max_order', 'loyalty_flag']].head(20)


,user_id,max_order,loyalty_flag
0,138,32,Regular customer
1,138,32,Regular customer
2,709,5,New customer
3,764,3,New customer
4,764,3,New customer
5,777,26,Regular customer
6,825,9,New customer
7,910,12,Regular customer
8,1052,20,Regular customer
9,1052,20,Regular customer


In [19]:
# 5 Get descriptive statistics of product prices by loyalty category
price_stats_loyalty = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

In [20]:
# results
price_stats_loyalty

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10284093.0,10.386336,327.996277,1.0,4.2,7.4,11.2,99999.0
New customer,6243990.0,13.294671,597.536255,1.0,4.2,7.4,11.3,99999.0
Regular customer,15876776.0,12.495718,539.700684,1.0,4.2,7.4,11.3,99999.0


In [22]:
 # descriptive statistics of prices show differences in spending habits by loyalty category. 
    #On average, new customers purchase slightly higher-priced products ($13.29) than regular customers ($12.49) and loyal customers ($10.38) 
        #This insight can help the marketing team target promotions and recommendations based on the price sensitivity of each customer.

In [23]:
#6 Create a column with the mean price of products purchased by each user
ords_prods_merge['mean_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

C:\Users\richa\AppData\Local\Temp\ipykernel_23008\1160380806.py:2: FutureWarning: The provided callable <function mean at 0x000001E772EAFB00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['mean_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [24]:
# Check the first 20 rows
ords_prods_merge[['user_id', 'prices', 'mean_price']].head(20)

,user_id,prices,mean_price
0,138,5.8,6.935811
1,138,5.8,6.935811
2,709,5.8,7.930208
3,764,5.8,4.972414
4,764,5.8,4.972414
5,777,5.8,6.935398
6,825,5.8,5.957576
7,910,5.8,6.680000
8,1052,5.8,7.162500
9,1052,5.8,7.162500


In [25]:
# 6 Assign spending flag based on mean_price
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spending_flag'] = 'High spender'

# Check the first 10 rows
ords_prods_merge[['user_id', 'mean_price', 'spending_flag']].head(10)


,user_id,mean_price,spending_flag
0,138,6.935811,Low spender
1,138,6.935811,Low spender
2,709,7.930208,Low spender
3,764,4.972414,Low spender
4,764,4.972414,Low spender
5,777,6.935398,Low spender
6,825,5.957576,Low spender
7,910,6.680000,Low spender
8,1052,7.162500,Low spender
9,1052,7.162500,Low spender


In [26]:
# 7 Create a column with the median days since prior order for each user
ords_prods_merge['median_days_since_prior'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

# Check first 10 rows
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_since_prior']].head(10)


C:\Users\richa\AppData\Local\Temp\ipykernel_23008\1732937793.py:2: FutureWarning: The provided callable <function median at 0x000001E772FEAC00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_prior'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


,user_id,days_since_prior_order,median_days_since_prior
0,138,3.0,8.0
1,138,20.0,8.0
2,709,6.0,8.0
3,764,NaN,9.0
4,764,9.0,9.0
5,777,26.0,11.0
6,825,30.0,20.0
7,910,30.0,6.0
8,1052,19.0,10.0
9,1052,15.0,10.0


In [30]:
# 7b Assign order frequency categories based on median days since prior order
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior'] > 10) & (ords_prods_merge['median_days_since_prior'] <= 20), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

# Check first 10 rows
ords_prods_merge[['user_id', 'median_days_since_prior', 'order_frequency_flag']].head(10)


,user_id,median_days_since_prior,order_frequency_flag
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
5,777,11.0,Regular customer
6,825,20.0,Regular customer
7,910,6.0,Frequent customer
8,1052,10.0,Frequent customer
9,1052,10.0,Frequent customer


In [32]:
print(ords_prods_merge.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   product_id               uint16  
 1   product_name             category
 2   aisle_id                 uint8   
 3   department_id            int8    
 4   prices                   float32 
 5   order_id                 int32   
 6   user_id                  int32   
 7   order_number             int8    
 8   order_dow                int8    
 9   order_hour_of_day        int8    
 10  days_since_prior_order   float32 
 11  add_to_cart_order        uint8   
 12  reordered                int8    
 13  _merge                   category
 14  merge_indicator          category
 15  price_range_loc          category
 16  Busiest day              category
 17  busiest_days_new         category
 18  busiest_period_of_day    category
 19  max_order                int8    
 20  loyalty_flag          

In [33]:
for col in ords_prods_merge.columns:
    if pd.api.types.is_integer_dtype(ords_prods_merge[col]):
        ords_prods_merge[col] = pd.to_numeric(
            ords_prods_merge[col], downcast='integer')
    elif pd.api.types.is_float_dtype(ords_prods_merge[col]):
        ords_prods_merge[col] = pd.to_numeric(
            ords_prods_merge[col], downcast='float')


In [34]:
print(ords_prods_merge.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 25 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   product_id               uint16  
 1   product_name             category
 2   aisle_id                 uint8   
 3   department_id            int8    
 4   prices                   float32 
 5   order_id                 int32   
 6   user_id                  int32   
 7   order_number             int8    
 8   order_dow                int8    
 9   order_hour_of_day        int8    
 10  days_since_prior_order   float32 
 11  add_to_cart_order        uint8   
 12  reordered                int8    
 13  _merge                   category
 14  merge_indicator          category
 15  price_range_loc          category
 16  Busiest day              category
 17  busiest_days_new         category
 18  busiest_period_of_day    category
 19  max_order                int8    
 20  loyalty_flag          

In [31]:
# Export the dataframe as a pickle file
ords_prods_merge.to_pickle(r'C:\Users\richa\Documents\01-2026 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_merged_flags.pkl')